### Sec. -1 Initiation


In [106]:
import os
import pandas as pd
from utils.config_helper import update_nested_toml, load_config
os.makedirs("logs", exist_ok=True)


In [107]:
# TYPE = input("Enter the type of the config file: ")
TYPE = "prostate"
CONFIG_PATH = f"../config/{TYPE}.toml"
config = load_config(CONFIG_PATH)

In [108]:
train_df_file = config["init"]["hyper"]["train_df_file"]
dmp_files = config["init"]["hyper"]["dmp_files"]
majority_out_path = config["init"]["hyper"]["majority_out_path"]

### Sec. 0 process norm examples


#### 1. Split Dataset Example


In [109]:
import json
from utils.process_norm import *

In [113]:
# read the json file
# cancer_type = input("Enter the cancer type: ")
cancer_type = "prostate"
with open(f"../config/{cancer_type}.json") as f:
    J = json.load(f)

In [115]:
# data_source = input("Enter the raw all_beta_normalized file: ")
data_source = "GDC_prostate_tissue"
df = pd.read_csv(J[data_source]['file'])


In [ ]:
ID = list(df.iloc[:, 0]) + ["label"]
sample_count = 2
normal_count = 50
tumor_count = 502
X = df.iloc[:, 1::sample_count].T
print(X.shape[0],normal_count,tumor_count)

1004 502 502


In [ ]:
dfo = organize_dataset(df, J[data_source]["normal"], J[data_source]["tumor"], J[data_source]["sample_count"])
complement_df, ratio_df = split_dataset(dfo, J[data_source]["split_test"], J[data_source]["random_state"])
complement_df.to_csv("all_beta_normalized_80.csv", index=False)
ratio_df.to_csv("all_beta_normalized_20.csv", index=False)

INFO complement_df feature: 373340
INFO complement_df sample (normal, tumor): (401, 402)
INFO ratio_df feature: 373340
INFO ratio_df sample (normal, tumor): (101, 100)


In [ ]:
ratio_df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,191,192,193,194,195,196,197,198,199,200
0,cg00000957,0.866258,0.848018,0.843079,0.830946,0.839486,0.807396,0.849108,0.858916,0.849391,...,0.809305,0.867759,0.880241,0.876851,0.874233,0.895410,0.850494,0.899322,0.880528,0.837177
1,cg00001349,0.852034,0.868409,0.854832,0.757838,0.832331,0.815491,0.731017,0.818800,0.734660,...,0.840027,0.793202,0.843819,0.811798,0.842095,0.802152,0.752804,0.866580,0.857917,0.792160
2,cg00001583,0.360523,0.103146,0.105064,0.050091,0.215267,0.136135,0.052770,0.224399,0.255017,...,0.133560,0.496975,0.481672,0.406983,0.625404,0.095873,0.276501,0.794870,0.838029,0.583207
3,cg00002028,0.020525,0.028840,0.025378,0.029126,0.025493,0.019812,0.027149,0.037735,0.036445,...,0.018602,0.014319,0.031543,0.035377,0.043325,0.016920,0.019389,0.046088,0.044256,0.025010
4,cg00002719,0.029307,0.051110,0.032113,0.035411,0.094550,0.047173,0.039447,0.088366,0.048843,...,0.345822,0.675067,0.696279,0.235251,0.574570,0.204156,0.025527,0.754098,0.850508,0.520944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373336,cg27657363,0.968167,0.957530,0.965772,0.946106,0.961166,0.964216,0.948009,0.966098,0.950632,...,0.934152,0.945786,0.897782,0.958972,0.944172,0.966352,0.958431,0.953648,0.920626,0.924836
373337,cg27657537,0.220613,0.168620,0.130816,0.109222,0.342180,0.232905,0.102387,0.313383,0.263290,...,0.411657,0.564375,0.665107,0.538359,0.610849,0.566487,0.153585,0.667166,0.729365,0.496844
373338,cg27662611,0.049505,0.044676,0.033138,0.058121,0.019983,0.029528,0.056332,0.068099,0.041418,...,0.036843,0.018571,0.035393,0.042450,0.064249,0.022017,0.044757,0.055987,0.042192,0.043591
373339,cg27665648,0.917473,0.920424,0.874715,0.879550,0.887804,0.899420,0.915299,0.917810,0.884160,...,0.813484,0.813176,0.897317,0.897426,0.870542,0.895949,0.878483,0.914036,0.875042,0.869568


#### 2. Merge Dataset Example


In [20]:
df0 = pd.read_csv(f"../stomach/champ_result/gdc_stomach_GSE99553/all_beta_normalized_0.csv")
df1 = pd.read_csv(f"../stomach/champ_result/gdc_stomach_GSE99553/all_beta_normalized_GSE99553.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../stomach/champ_result/gdc_stomach_GSE99553/all_beta_normalized_0.csv'

In [4]:
df0 = organize_dataset(df0, 2, 395, 2)
df1 = organize_dataset(df1, 84, 0, 1)

In [ ]:
merged_df = merge_datasets(df0, df1)
merged_df.to_csv("merged.csv", index=False)

#### 3. Inspect NaN Example


In [21]:
# Example usage:
df = pd.DataFrame({
    'ID': ['A1', 'A2', 'A3', 'A4', 'label'],
    'Col1': [1, 2, None, 4, 1],
    'Col2': [5, None, 7, 8, 1],
    'Col3': [9, 10, 11, 12, 0]
})

# Column-wise check
print(inspect_nan(df, mode="column"))

# Row-wise check
print(inspect_nan(df, mode="row"))

INFO Columns with NaNs: {'Col1': np.float64(1.0), 'Col2': np.float64(1.0)}
      ID  Col1  Col2  Col3
0     A1   1.0   5.0     9
1     A2   2.0   NaN    10
2     A3   NaN   7.0    11
3     A4   4.0   8.0    12
4  label   1.0   1.0     0
INFO Rows with NaNs: ['A2', 'A3']
      ID  Col1  Col2  Col3
0     A1   1.0   5.0     9
1     A2   2.0   NaN    10
2     A3   NaN   7.0    11
3     A4   4.0   8.0    12
4  label   1.0   1.0     0


### Sec. 1 Delta Beta Calculation


In [5]:
os.makedirs(f"{majority_out_path}/section_1", exist_ok=True)

In [6]:
train_df = pd.read_csv(train_df_file)

In [7]:
# remove outlier in terms of every column
def IQR(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    upper_fence = Q3 + IQR * 1.5
    lower_fence = Q1 - IQR * 1.5
    return upper_fence, lower_fence


def no_outlier(df):
    upper_fence, lower_fence = IQR(df)
    ddf = df[(df > lower_fence) & (df < upper_fence)]
    return ddf

In [8]:
# get normal count by the count of 0 in the last row
normal_count = int((train_df.iloc[-1, 1:] == 0).sum())
all_beta_normalized_normal = train_df.iloc[:-1, 1 : normal_count + 1 :].T

all_beta_normalized_tumor = train_df.iloc[:-1, normal_count + 1 : :].T

In [9]:
all_beta_normalized_normal = no_outlier(all_beta_normalized_normal)
all_beta_normalized_tumor = no_outlier(all_beta_normalized_tumor)

In [10]:
train_normal_avg = all_beta_normalized_normal.mean(skipna=True, axis=0)

In [11]:
all_beta_normalized_tumor = all_beta_normalized_tumor.subtract(
    train_normal_avg, axis=1
)

In [13]:
all_beta_normalized_tumor = no_outlier(all_beta_normalized_tumor)

In [14]:
train_tumor_mean = all_beta_normalized_tumor.mean(skipna=True, axis=0)

In [15]:
delta_beta = pd.merge(
    train_df.iloc[:-1, :1],
    pd.DataFrame(train_tumor_mean, columns=["dbeta"]),
    left_index=True,
    right_index=True,
)


In [16]:
# record the list of feature with dbeta being NaN
update_nested_toml("preprocess.dbeta", "delta_beta_avg_feature_num", delta_beta.shape[0])
update_nested_toml(
    "preprocess.dbeta",
    "NaN_dbeta_feature",
    delta_beta.loc[pd.isna(delta_beta["dbeta"]), "Unnamed: 0"].tolist(),
)
delta_beta = delta_beta.dropna(axis=0)
update_nested_toml("preprocess.dbeta", "delta_beta_avg_feature_num_remove_NaN", delta_beta.shape[0])

In [17]:
dmp = pd.read_csv("../prostate/champ_result/GDC_prostate_tissue/DMP_result_0.csv")
dmp = dmp[["Unnamed: 0", "gene", "feature"]]

In [18]:
update_nested_toml("preprocess.dbeta", "dmp_before_dropna_shape_feature", dmp.shape[0])

dmp = dmp.dropna(axis=0)

update_nested_toml("preprocess.dbeta", "dmp_after_dropna_shape_feature", dmp.shape[0])

In [19]:
result = pd.merge(delta_beta, dmp, on="Unnamed: 0", how="left")
update_nested_toml(
    "preprocess.dbeta", "delta_beta_avg_feature_num_remove_NaN_join_dmp", result.shape[0]
)

In [20]:
def find_max_dBeta_grouped(group):
    idx_max = group["dbeta"].abs().idxmax()
    return group.loc[idx_max]


dbeta_info = result.groupby("gene", as_index=False).apply(
    find_max_dBeta_grouped, include_groups=False
)

In [21]:
dbeta_info.columns = ["gene", "ID", "dbeta", "feature"]
dbeta_info = dbeta_info[["ID", "gene", "dbeta", "feature"]]

In [22]:
dbeta_info

,ID,gene,dbeta,feature
0,cg03630821,A1BG,0.148724,Body
1,cg27394794,A1CF,-0.337929,Body
2,cg07196505,A2BP1,-0.435004,Body
3,cg00336946,A2LD1,-0.246354,TSS1500
4,cg00134295,A2M,-0.197167,TSS1500
...,...,...,...,...
18647,cg23995459,ZYG11B,-0.191031,TSS1500
18648,cg18074834,ZYX,0.036728,Body
18649,cg16463044,ZZEF1,-0.156933,Body
18650,cg22221847,ZZZ3,-0.032673,3'UTR


In [ ]:
# comorbidity = pd.read_csv(
#     "../external_result/matchgene174_single_3Y10__OR2.txt", sep="\t", header=None
# )
# dbeta = dbeta[
#     dbeta["gene"].isin(comorbidity[0])
# ]

# result_max_per_gene_single

In [23]:
dbeta_info["dbeta"] = dbeta_info["dbeta"].apply(lambda x: round(x, 6))
dbeta_info.to_csv(f"{majority_out_path}/section_1/dbeta.csv", index=False)

### Sec. 2 Filter Genes by Average Delta Beta Values


#### 2.1 Filtering TSS


In [24]:
os.makedirs(f"{majority_out_path}/section_2", exist_ok=True)

In [ ]:
# dbeta_info = pd.read_csv(f"{majority_out_path}/dbeta.csv")

In [25]:
TSS = dbeta_info[dbeta_info["feature"].str.contains("TSS")]

In [26]:
TSS.to_csv(f"{majority_out_path}/section_2/dbeta_TSS.csv", index=False)

#### 2.2 Thresholding


In [27]:
threshold = 1
dbeta_TSS_threshold = TSS[abs(TSS["dbeta"]) > threshold]
while True:
    dbeta_TSS_threshold = TSS[abs(TSS["dbeta"]) > threshold]
    count = dbeta_TSS_threshold.shape[0]
    if (
        config["preprocess"]["filtering"]["hyper"]["avg_dbeta_lower_bound"]
        <= count
        <= config["preprocess"]["filtering"]["hyper"]["avg_dbeta_upper_bound"]
    ):
        break
    threshold -= 0.01
threshold = round(threshold, 2)
update_nested_toml("preprocess.filtering", "threshold", threshold)

In [28]:
dbeta_TSS_threshold.to_csv(f"{majority_out_path}/section_2/dbeta_TSS_{threshold}.csv", index=False)

#### 2.3 Visualization


In [29]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.kdeplot(dbeta_TSS_threshold["dbeta"])
plt.xlabel("delta Beta value")
plt.title("Density plot of delta Beta value")
plt.savefig(f"{majority_out_path}/section_2/dbeta_TSS_{threshold}.png")
plt.close()

In [ ]:
# train_df = pd.read_csv(train_df_file)

In [30]:
train_df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,793,794,795,796,797,798,799,800,801,802
0,cg00000957,0.824770,0.838333,0.875196,0.810238,0.860323,0.884161,0.864832,0.863311,0.861795,...,0.833749,0.886146,0.832414,0.892412,0.891901,0.905091,0.835944,0.745082,0.884742,0.869875
1,cg00001349,0.827872,0.875523,0.810576,0.806504,0.821141,0.852266,0.846344,0.715566,0.801538,...,0.605978,0.900709,0.819733,0.905889,0.931324,0.947951,0.130110,0.801109,0.833059,0.840674
2,cg00001583,0.121659,0.520205,0.241731,0.134270,0.351876,0.360133,0.258585,0.095149,0.048460,...,0.128480,0.286445,0.421377,0.622970,0.439232,0.783077,0.473451,0.067169,0.208544,0.460007
3,cg00002028,0.042886,0.034299,0.021822,0.020703,0.024398,0.017322,0.032324,0.028100,0.027861,...,0.040477,0.033830,0.043285,0.026778,0.023449,0.021500,0.034437,0.025156,0.024711,0.065117
4,cg00002719,0.065884,0.490811,0.069449,0.047640,0.212139,0.024730,0.206436,0.032892,0.048767,...,0.090535,0.376104,0.235849,0.807773,0.753019,0.814458,0.313751,0.151836,0.192325,0.494103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373336,cg27657363,0.945175,0.931059,0.957842,0.963020,0.966025,0.968234,0.967821,0.927369,0.951071,...,0.899267,0.960262,0.915603,0.946284,0.967195,0.916364,0.923337,0.950343,0.960460,0.902534
373337,cg27657537,0.298178,0.282966,0.245997,0.220096,0.283525,0.185075,0.381661,0.249224,0.101864,...,0.183660,0.427111,0.451669,0.736287,0.657288,0.753574,0.441521,0.151265,0.469957,0.293109
373338,cg27662611,0.068914,0.033763,0.043791,0.031257,0.060840,0.031299,0.067576,0.065646,0.063771,...,0.061230,0.030288,0.050947,0.065393,0.077492,0.064744,0.029955,0.055340,0.045296,0.052520
373339,cg27665648,0.858845,0.824831,0.880827,0.896936,0.942436,0.915946,0.922258,0.893683,0.925910,...,0.857805,0.918376,0.871963,0.753748,0.872375,0.925096,0.846256,0.802507,0.810491,0.906675


In [31]:
beta_df = train_df.iloc[:-1, :]
beta_df = beta_df[beta_df["Unnamed: 0"].isin(dbeta_info["ID"])]
X = beta_df.iloc[:, 1:].dropna(axis=0).T
y = train_df.iloc[-1, 1:].astype(int).to_list()

In [32]:
# DEBUG
print(f"X shape: {X.shape}")
print(f"y shape: {len(y)}")
# END

X shape: (803, 18652)
y shape: 803


In [33]:
import plotly.express as px
import pandas as pd
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

df = pd.DataFrame(
    {
        "Principal Component 1": X_pca[:, 0],
        "Principal Component 2": X_pca[:, 1],
        "Principal Component 3": X_pca[:, 2],
        "Class": y,
    }
)
fig = px.scatter_3d(
    df,
    x="Principal Component 1",
    y="Principal Component 2",
    z="Principal Component 3",
    color="Class",
    title="PCA of Dataset",
    color_continuous_scale="Viridis",
)

fig.update_layout(
    scene=dict(
        xaxis_title="Principal Component 1",
        yaxis_title="Principal Component 2",
        zaxis_title="Principal Component 3",
    )
)

fig.write_html(f"{majority_out_path}/section_2/preprocess_filtering_pca.html")


#### remember to run clustering on the filtered genes first continue to the next step


### Sec. 3 Feature Selection with ML (SFS)

sequential forward selection


#### Remove previous results

Warning: This step is not reversible


In [9]:
import shutil
import os
if os.path.exists(f"{majority_out_path}/sfs"):
    shutil.rmtree(f"{majority_out_path}/sfs")


#### 3.1 Preparation(SFS)


In [10]:
config = load_config(CONFIG_PATH)

train_out_path = config["feature_selection"]["hyper"]["train_out_path"]
validate_out_path = config["feature_selection"]["hyper"]["validate_out_path"]

In [11]:
os.makedirs(f"{train_out_path}", exist_ok=True)
os.makedirs(f"{validate_out_path}", exist_ok=True)

In [12]:
config = load_config(CONFIG_PATH)
dbeta_info_file = config["feature_selection"]["hyper"]["dbeta_info_file"]
dbeta_info = pd.read_csv(f"{majority_out_path}/{dbeta_info_file}")

FileNotFoundError: [Errno 2] No such file or directory: '../prostate/result/GSE269244/split80/../prostate/result/GSE269244/split80/section_5/dbeta_TSS_0.21_consensus.csv'

In [8]:
# check if logs/ folder exists
os.makedirs("logs", exist_ok=True)
from utils.train_helper import TrainHelper

In [ ]:
del th

In [14]:
# note that there is setup_dbeta in TrainHelper to further cut down the feature size
th = TrainHelper(dbeta_info)

#### 3.2 Selection(SFS)


In [61]:
train_df = pd.read_csv(f"{majority_out_path}/{majority_df_path}")
validate_df = pd.read_csv(f"{minority_out_path}/{minority_df_path}")
th.set_train_validate_df(train_df, validate_df)

NameError: name 'majority_df_path' is not defined

In [15]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

selection_models = {
    "SVM": SVC(kernel="linear", random_state=42),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "RandomForest": RandomForestClassifier(
        random_state=42,
        n_estimators=10,
    ),
    "XGBoost": XGBClassifier(
        random_state=42,
        n_estimators=10,
        ),
}

In [12]:
th.set_selection_models(selection_models)
th.set_train_validate()

In [ ]:
os.makedirs(f"{majority_out_path}/sfs", exist_ok=True)

th.select_feature_sfs(
    out_path = f"{majority_out_path}/sfs/selected_feature.txt",
    step= 4,
    n_features_to_select="cluster"
)

### Sec. 3 Feature Selection with ML (RFE)

recursive feature elimination


#### Remove previous results

Warning: This step is not reversible


In [16]:
# import shutil
# import os
# if os.path.exists(f"{majority_out_path}/section_3/rfe"):
#     shutil.rmtree(f"{majority_out_path}/section_3/rfe")
# if os.path.exists(f"{minority_out_path}/section_3/rfe"):
#     shutil.rmtree(f"{minority_out_path}/section_3/rfe")

#### 3.1 Preparation


In [62]:
config = load_config(CONFIG_PATH)

train_out_path = config["feature_selection"]["hyper"]["train_out_path"]
validate_out_path = config["feature_selection"]["hyper"]["validate_out_path"]
training_param_file = config["feature_selection"]["hyper"]["training_param_file"]

In [63]:
os.makedirs(f"{train_out_path}", exist_ok=True)
os.makedirs(f"{validate_out_path}", exist_ok=True)

In [64]:
dbeta_info_file = config["feature_selection"]["hyper"]["dbeta_info_file"]

TSS_threshold = pd.read_csv(dbeta_info_file)

In [65]:
from utils.train_helper import TrainHelper

In [66]:
# note that there is setup_dbeta in TrainHelper to further cut down the feature size
th = TrainHelper(TSS_threshold)

#### 3.2 Selection


In [67]:
from utils.process_norm import *

In [68]:
train_df = pd.read_csv(train_df_file)
validate_df_file = config["feature_selection"]["hyper"]["validate_df_file"]
validate_df = pd.read_csv(validate_df_file)

In [ ]:
# train_df = inspect_nan(train_df, mode="column", remove=True)

In [69]:
inspect_nan(train_df, mode="column")
inspect_nan(validate_df, mode="column")

INFO Columns with NaNs: {}
INFO Columns with NaNs: {}


,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,191,192,193,194,195,196,197,198,199,200
0,cg00000957,0.866258,0.848018,0.843079,0.830946,0.839486,0.807396,0.849108,0.858916,0.849391,...,0.809305,0.867759,0.880241,0.876851,0.874233,0.895410,0.850494,0.899322,0.880528,0.837177
1,cg00001349,0.852034,0.868409,0.854832,0.757838,0.832331,0.815491,0.731017,0.818800,0.734660,...,0.840027,0.793202,0.843819,0.811798,0.842095,0.802152,0.752804,0.866580,0.857917,0.792160
2,cg00001583,0.360523,0.103146,0.105064,0.050091,0.215267,0.136135,0.052770,0.224399,0.255017,...,0.133560,0.496975,0.481672,0.406983,0.625404,0.095873,0.276501,0.794870,0.838029,0.583207
3,cg00002028,0.020525,0.028840,0.025378,0.029126,0.025493,0.019812,0.027149,0.037735,0.036445,...,0.018602,0.014319,0.031543,0.035377,0.043325,0.016920,0.019389,0.046088,0.044256,0.025010
4,cg00002719,0.029307,0.051110,0.032113,0.035411,0.094550,0.047173,0.039447,0.088366,0.048843,...,0.345822,0.675067,0.696279,0.235251,0.574570,0.204156,0.025527,0.754098,0.850508,0.520944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373336,cg27657363,0.968167,0.957530,0.965772,0.946106,0.961166,0.964216,0.948009,0.966098,0.950632,...,0.934152,0.945786,0.897782,0.958972,0.944172,0.966352,0.958431,0.953648,0.920626,0.924836
373337,cg27657537,0.220613,0.168620,0.130816,0.109222,0.342180,0.232905,0.102387,0.313383,0.263290,...,0.411657,0.564375,0.665107,0.538359,0.610849,0.566487,0.153585,0.667166,0.729365,0.496844
373338,cg27662611,0.049505,0.044676,0.033138,0.058121,0.019983,0.029528,0.056332,0.068099,0.041418,...,0.036843,0.018571,0.035393,0.042450,0.064249,0.022017,0.044757,0.055987,0.042192,0.043591
373339,cg27665648,0.917473,0.920424,0.874715,0.879550,0.887804,0.899420,0.915299,0.917810,0.884160,...,0.813484,0.813176,0.897317,0.897426,0.870542,0.895949,0.878483,0.914036,0.875042,0.869568


In [70]:
th.set_train_validate_df(train_df, validate_df)

In [71]:
from utils.train_helper import set_parameters
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

import json

with open(training_param_file, "r") as f:
    training_param = json.load(f)
xgb_grid = set_parameters(XGBClassifier(random_state=42), training_param["XGBoost"])
rf_grid = set_parameters(
    RandomForestClassifier(random_state=42), training_param["RandomForest"]
)
svm_grid = set_parameters(SVC(random_state=42, probability=True), training_param["SVM"])
dt_grid = set_parameters(
    DecisionTreeClassifier(random_state=42), training_param["DecisionTree"]
)

train_models = {
    "XGBoost": xgb_grid,
    "RandomForest": rf_grid,
    "SVM": svm_grid,
    "DecisionTree": dt_grid,
}

In [72]:
selection_models = {
    "SVM": SVC(kernel="linear", random_state=42),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "RandomForest": RandomForestClassifier(
        random_state=42,
        n_estimators=10,
    ),
    "XGBoost": XGBClassifier(
        random_state=42,
        n_estimators=10,
        ),
}

In [73]:
th.set_selection_models(selection_models)
th.set_grid_estimators(train_models)
th.set_train_validate()

In [74]:
th.select_feature_rfe(
    train_out_path = train_out_path,
    validate_out_path = validate_out_path,
    selected_feature_path = f"{train_out_path}/selected_feature.txt",
    feature_range = (1, 6, 1),
)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Fitting 5 fo

D:\Users\005535\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\ma\core.py:2892: RuntimeWarning:

invalid value encountered in cast



Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 18 candidates, totalling 90 fits


### Sec. 4 Clean Selected Features


#### 4.1 Generate Feature json for SimpleModel


In [79]:
from utils.train_helper import read_selected_features, read_selected_features_json, TrainHelper
config = load_config(CONFIG_PATH)
dbeta_info_file = config["machine_learning"]["hyper"]["dbeta_info_file"]
dbeta_info = pd.read_csv(f"{majority_out_path}/{dbeta_info_file}")
th = TrainHelper(dbeta_info)

folder = "sfs"
features = read_selected_features(f"{majority_out_path}/{folder}/selected_feature.txt")
th.generate_selected_features(
    features,
    f"{majority_out_path}/{folder}/selected_features.json",
    mode="min",
    out_format="json",
)

# use this to read json
read_selected_features_json(f"{majority_out_path}/{folder}/selected_features.json")

FileNotFoundError: [Errno 2] No such file or directory: '../prostate/result/GDC_prostate_tissue/split80/sfs/selected_feature.txt'

#### 4.2 Gather Selected gene list from best selection model


In [80]:
rfe_train = pd.read_csv(f"{train_out_path}/rfe.csv")
rfe_validate = pd.read_csv(f"{validate_out_path}/rfe.csv")
fpr_tpr_train = pd.read_csv(f"{train_out_path}/roc_curve.csv")
fpr_tpr_validate = pd.read_csv(f"{validate_out_path}/roc_curve.csv")
rfe_j = pd.merge(rfe_train, rfe_validate, on=["selection_model", "train_model", "features"], suffixes=('_train', '_validate'))
fpr_tpr_j = pd.merge(fpr_tpr_train, fpr_tpr_validate, on=["selection_model", "train_model", "features"], suffixes=('_train', '_validate'))
J = pd.merge(rfe_j, fpr_tpr_j, on=["selection_model", "train_model", "features"])

In [81]:
import ast

J["fpr_train"] = J["fpr_train"].apply(ast.literal_eval)
J["tpr_train"] = J["tpr_train"].apply(ast.literal_eval)
J["fpr_validate"] = J["fpr_validate"].apply(ast.literal_eval)
J["tpr_validate"] = J["tpr_validate"].apply(ast.literal_eval)

In [82]:
from utils.painter import plot_roc_curve, create_performance_barchart

In [83]:
J['accuracy_diff'] = J['accuracy_train'] - J['accuracy_validate']
J['recall_diff'] = J['recall_train'] - J['recall_validate']
J['f1_score_diff'] = J['f1_score_train'] - J['f1_score_validate']
J['AUC_diff'] = J['AUC_train'] - J['AUC_validate']
J['MCC_diff'] = J['MCC_train'] - J['MCC_validate']
J['fbeta2_score_diff'] = J['fbeta2_score_train'] - J['fbeta2_score_validate']

In [84]:
# tweakable width and height
plot_roc_curve(
    J,
    "ROC Curves on Training Set",
    f"{train_out_path}/roc_curve.html",
    x_column = "fpr_train",
    y_column = "tpr_train",
    trace_name = ["selection_model", "train_model", "features"],
)
# tweakable width and height
plot_roc_curve(
    J,
    "ROC Curves on Testing Set",
    f"{validate_out_path}/roc_curve.html",
    x_column = "fpr_validate",
    y_column = "tpr_validate",
    trace_name = ["selection_model", "train_model", "features"],
)

ROC curve saved to ../prostate/result/GDC_prostate_tissue/split80/section_3/rfe/roc_curve.html
ROC curve saved to ../prostate/result/GDC_prostate_tissue/split20/section_3/rfe/roc_curve.html


In [85]:
# plot difference
performance_metrics = ['accuracy_diff', 'recall_diff', 'f1_score_diff', 'AUC_diff', 'MCC_diff', 'fbeta2_score_diff']
ground_by_train_model = J.groupby('train_model')[performance_metrics].mean()
ground_by_train_model['train_model'] = ground_by_train_model.index
ground_by_train_model.to_csv(f"{validate_out_path}/performance_diff_grouped_by_train_model.csv", index=False)
color_mapping = {
    "accuracy_diff": "blue",
    "recall_diff": "red",
    "f1_score_diff": "green",
    "AUC_diff": "purple",
    "MCC_diff": "orange",
    "fbeta2_score_diff": "brown",
}
create_performance_barchart(
    df=ground_by_train_model,
    color_mapping=color_mapping,
    metric="train_model",
    out_path=f"{validate_out_path}/performance_diff_grouped_by_train_model.html",
    title="Grouped Performance Difference between Training and Testing Set",
    x_axis_label="Performance Difference (Training - Testing)",
    y_axis_label="Train Model",
    orientation="h",
)

Performance difference saved to ../prostate/result/GDC_prostate_tissue/split20/section_3/rfe/performance_diff_grouped_by_train_model.html


In [86]:
J = J[["selection_model", "train_model", "features", "accuracy_validate", "recall_validate", "f1_score_validate", "AUC_validate", "MCC_validate", "fbeta2_score_validate"]]

In [87]:
# group by train_model, for each train_model, calculate the mean of each performance metric
performance_metrics = ['accuracy_validate', 'recall_validate',
                       'f1_score_validate', 'AUC_validate', 'MCC_validate']
ground_by_train_model = J.groupby('train_model')[performance_metrics].mean()
ground_by_train_model['train_model'] = ground_by_train_model.index
ground_by_train_model.to_csv(
    f"{validate_out_path}/performance_metrics_grouped_by_train_model.csv", index=False)
color_mapping = {
    "accuracy_validate": "blue",
    "recall_validate": "red",
    "f1_score_validate": "green",
    "AUC_validate": "purple",
    "MCC_validate": "orange",
}
create_performance_barchart(
    df=ground_by_train_model,
    color_mapping=color_mapping,
    metric="train_model",
    out_path=f"{validate_out_path}/performance_metrics_grouped_by_train_model.html",
    title="Grouped Performance Metrics by Train Model",
    x_axis_label="Performance",
    y_axis_label="Train Model",
    orientation="h",
)
best_train_model = ground_by_train_model['MCC_validate'].idxmax()
print(f"Best train model: {best_train_model}")
ground_by_feature = J[J['train_model'] == best_train_model].groupby('features')[
    performance_metrics].mean()
ground_by_feature['features'] = ground_by_feature.index
ground_by_feature.to_csv(
    f"{validate_out_path}/performance_metrics_grouped_by_feature.csv", index=False)
create_performance_barchart(
    df=ground_by_feature,
    color_mapping=color_mapping,
    metric="features",
    out_path=f"{validate_out_path}/performance_metrics_grouped_by_feature.html",
    title="Grouped Performance Metrics by Feature",
    x_axis_label="Performance",
    y_axis_label="Feature",
    orientation="h",
)
best_num_of_feature = ground_by_feature['MCC_validate'].idxmax()
print(f"Best number of feature: {best_num_of_feature}")
best_performance_records = J[(J['train_model'] == best_train_model) & (
    J['features'] == best_num_of_feature)]
best_performance_records.to_csv(
    f"{validate_out_path}/best_performance_records.csv", index=False)

Performance difference saved to ../prostate/result/GDC_prostate_tissue/split20/section_3/rfe/performance_metrics_grouped_by_train_model.html
Best train model: SVM
Performance difference saved to ../prostate/result/GDC_prostate_tissue/split20/section_3/rfe/performance_metrics_grouped_by_feature.html
Best number of feature: 2


In [88]:
from utils.train_helper import read_selected_features, read_selected_features_json, TrainHelper
config = load_config(CONFIG_PATH)
dbeta_info_file = config["feature_selection"]["hyper"]["dbeta_info_file"]
dbeta_info = pd.read_csv(dbeta_info_file)
th = TrainHelper(dbeta_info)

features = read_selected_features(f"{train_out_path}/selected_feature.txt")

th.generate_selected_features(
    features,
    f"{validate_out_path}/selected_features.json",
    mode=int(best_num_of_feature),
    out_format="json",
)

# use this to read json
read_selected_features_json(f"{validate_out_path}/selected_features.json")

defaultdict(list,
            {'best': ['OR4K2',
              'RNF152',
              'HAAO',
              'GCM2',
              'CDX2',
              'RARRES1',
              'ITIH5']})

### Sec. 5 Clustering Visualization


#### 1. load data

remember to calculate distance matrix first


In [40]:
import pandas as pd
import numpy as np
from utils.clustering_helper import hierarchical_clustering, check_distance_matrix

In [41]:
result_prefix = config["clustering_visual"]["hyper"]["result_prefix"]
dbeta_file = config["clustering_visual"]["hyper"]["dbeta_file"]
bp_file = config["clustering_visual"]["hyper"]["bp_file"]
cc_file = config["clustering_visual"]["hyper"]["cc_file"]
mf_file = config["clustering_visual"]["hyper"]["mf_file"]
terms_count_file = config["clustering_visual"]["hyper"]["terms_count_file"]
result_out_path = config["clustering_visual"]["hyper"]["result_out_path"]

In [42]:
os.makedirs(result_out_path, exist_ok=True)

In [43]:
gene_set = pd.read_csv(dbeta_file, index_col=0)
distance_matrix_bp = pd.read_csv(bp_file, index_col=0)
distance_matrix_cc = pd.read_csv(cc_file, index_col=0)
distance_matrix_mf = pd.read_csv(mf_file, index_col=0)
terms_count = pd.read_csv(terms_count_file, index_col=0)

In [44]:
# replace NaN with 0
distance_matrix_bp = distance_matrix_bp.fillna(0)
distance_matrix_cc = distance_matrix_cc.fillna(0)
distance_matrix_mf = distance_matrix_mf.fillna(0)

In [45]:
# reindex distance matrix
index_bp = distance_matrix_bp.index
index_cc = distance_matrix_cc.index
index_mf = distance_matrix_mf.index
index = index_bp.union(index_cc).union(index_mf)
distance_matrix_bp_ = distance_matrix_bp.reindex(index=index, columns=index, fill_value=0)
distance_matrix_cc_ = distance_matrix_cc.reindex(index=index, columns=index, fill_value=0)
distance_matrix_mf_ = distance_matrix_mf.reindex(index=index, columns=index, fill_value=0)

In [46]:
# make a array of distance matrix for each ontology
distance_matrix = []

distance_matrix.append(distance_matrix_bp_)
distance_matrix.append(distance_matrix_cc_)
distance_matrix.append(distance_matrix_mf_)

#### 2. Weighted Sum


In [47]:
weight = [count for count in terms_count["count"]]
weight = weight / np.sum(weight)
masks = np.array([~np.isnan(distance_matrix[i].values) for i in range(3)])

valid_weights = np.array([weight[i] for i in range(3)])[:, None, None] * masks

weight_sums = valid_weights.sum(axis=0)

normalized_weights = np.divide(valid_weights, weight_sums, where=weight_sums != 0)
weighted_sum = sum(
    np.nan_to_num(distance_matrix[i].values) * normalized_weights[i] for i in range(3)
)


weighted_sum_dataframe = pd.DataFrame(weighted_sum, index=index, columns=index)

weighted_sum_dataframe.head()

,ACE,ACOT11,ACSF2,ADAM11,ADAMTS12,ADCY1,ADCY4,AFAP1L1,ALDH1L1,ALOX12,...,WDR69,WFDC5,WIPF1,WNT16,ZFP42,ZNF167,ZNF48,ZNF577,ZNF655,ZP4
ACE,0.000000,0.690013,0.805765,0.637504,0.597412,0.660149,0.724040,0.056375,0.751434,0.697314,...,0.673222,0.721243,0.728313,0.706338,0.729743,0.617973,0.785282,0.617973,0.788893,0.745105
ACOT11,0.690013,0.000000,0.454229,0.655252,0.694421,0.635476,0.496537,0.041854,0.644318,0.510734,...,0.715572,0.705818,0.682993,0.687805,0.755394,0.718020,0.752457,0.718020,0.731587,0.739752
ACSF2,0.805765,0.454229,0.000000,0.822319,0.790916,0.756245,0.677267,0.059127,0.623949,0.519881,...,0.775754,0.796536,0.731077,0.823078,0.780517,0.706313,0.713800,0.706313,0.738220,0.805920
ADAM11,0.637504,0.655252,0.822319,0.000000,0.587065,0.704027,0.631173,0.178823,0.793147,0.756452,...,0.712262,0.746492,0.772532,0.716756,0.823593,0.797796,0.852584,0.797796,0.841362,0.761520
ADAMTS12,0.597412,0.694421,0.790916,0.587065,0.000000,0.674923,0.651679,0.056470,0.743394,0.700046,...,0.650572,0.689017,0.668338,0.633971,0.705853,0.692266,0.754840,0.692266,0.742766,0.679116


In [48]:
cluster_result_weighted = hierarchical_clustering(
    weighted_sum_dataframe,
    range_min=2,
    range_max=4,
    cluster_number=3,
    out_path=f"{result_out_path}/hierarchical_clustering_weighted_sum.png",
)

chosen number of clusters: 3


In [67]:
cluster_result_weighted.head()

,gene,cluster
0,ACCS,3
1,ACE,2
2,ACOX2,3
3,ACSF2,3
4,ACSS3,3


#### 3. Simple average


In [49]:
weight = [1, 1, 1]
masks = np.array([~np.isnan(distance_matrix[i].values) for i in range(3)])
valid_weights = np.array([weight[i] for i in range(3)])[:, None, None] * masks
weight_sums = valid_weights.sum(axis=0)
normalized_weights = np.divide(valid_weights, weight_sums, where=weight_sums != 0)
weighted_sum = sum(
    np.nan_to_num(distance_matrix[i].values) * normalized_weights[i] for i in range(3)
)
simple_sum_dataframe = pd.DataFrame(weighted_sum, index=index, columns=index)
simple_sum_dataframe.head()

,ACE,ACOT11,ACSF2,ADAM11,ADAMTS12,ADCY1,ADCY4,AFAP1L1,ALDH1L1,ALOX12,...,WDR69,WFDC5,WIPF1,WNT16,ZFP42,ZNF167,ZNF48,ZNF577,ZNF655,ZP4
ACE,0.000000,0.603333,0.727000,0.558000,0.518333,0.577000,0.597333,0.198000,0.651667,0.574000,...,0.598667,0.497000,0.645000,0.631667,0.674333,0.563000,0.645667,0.563000,0.683333,0.606667
ACOT11,0.603333,0.000000,0.442000,0.670667,0.635000,0.581667,0.490333,0.147000,0.476333,0.441667,...,0.619667,0.593000,0.559333,0.578333,0.635667,0.625333,0.636000,0.625333,0.571000,0.659333
ACSF2,0.727000,0.442000,0.000000,0.802667,0.739000,0.689333,0.605333,0.207667,0.527667,0.568000,...,0.700000,0.740333,0.634333,0.723000,0.684000,0.573667,0.569333,0.573667,0.598667,0.738000
ADAM11,0.558000,0.670667,0.802667,0.000000,0.509333,0.659333,0.581667,0.367667,0.761667,0.683667,...,0.695333,0.714667,0.729000,0.699667,0.786333,0.787333,0.811667,0.787333,0.781667,0.614667
ADAMTS12,0.518333,0.635000,0.739000,0.509333,0.000000,0.614667,0.563667,0.198333,0.665333,0.620667,...,0.576667,0.507667,0.613000,0.529333,0.637667,0.636000,0.665333,0.636000,0.640333,0.481000


In [50]:
cluster_result_simple = hierarchical_clustering(
    simple_sum_dataframe,
    range_min=2,
    range_max=4,
    cluster_number=3,
    out_path=f"{result_out_path}/hierarchical_clustering_simple_sum.png",
)

chosen number of clusters: 3


In [51]:
cluster_result_simple.head()

,gene,cluster
0,ACE,3
1,ACOT11,3
2,ACSF2,3
3,ADAM11,3
4,ADAMTS12,3


#### 4. Consensus clustering


In [52]:
cluster_bp = hierarchical_clustering(
    distance_matrix_bp, out_path=f"{result_out_path}/hierarchical_clustering_bp.png"
)
cluster_cc = hierarchical_clustering(
    distance_matrix_cc, out_path=f"{result_out_path}/hierarchical_clustering_cc.png"
)
cluster_mf = hierarchical_clustering(
    distance_matrix_mf, out_path=f"{result_out_path}/hierarchical_clustering_mf.png"
)

Best number of clusters: 31
Best number of clusters: 3
Best number of clusters: 2


In [53]:
cluster_bp.columns = ["gene", "cluster_bp"]
cluster_cc.columns = ["gene", "cluster_cc"]
cluster_mf.columns = ["gene", "cluster_mf"]
cluster_bp_cc = pd.merge(cluster_bp, cluster_cc, on="gene", how="outer")
cluster_go = pd.merge(cluster_bp_cc, cluster_mf, on="gene", how="outer")
cluster_go = cluster_go.fillna(-1)
print(cluster_go.shape)
cluster_go.head()

(316, 4)


,gene,cluster_bp,cluster_cc,cluster_mf
0,ACE,13.0,3.0,2.0
1,ACOT11,24.0,1.0,2.0
2,ACSF2,24.0,1.0,2.0
3,ADAM11,13.0,3.0,2.0
4,ADAMTS12,13.0,2.0,2.0


In [54]:
num_genes = cluster_go.shape[0]
consensus_matrix = np.zeros((num_genes, num_genes))
for i in range(num_genes):
    for j in range(i, num_genes):
        if cluster_go.iloc[i]["cluster_bp"] == cluster_go.iloc[j]["cluster_bp"]:
            consensus_matrix[i][j] += 1

        if cluster_go.iloc[i]["cluster_cc"] == cluster_go.iloc[j]["cluster_cc"]:
            consensus_matrix[i][j] += 1

        if cluster_go.iloc[i]["cluster_mf"] == cluster_go.iloc[j]["cluster_mf"]:
            consensus_matrix[i][j] += 1

consensus_matrix = pd.DataFrame(
    consensus_matrix, index=cluster_go["gene"], columns=cluster_go["gene"]
)
consensus_matrix += consensus_matrix.T
distance_matrix_consensus = 1 - consensus_matrix / 3
np.fill_diagonal(distance_matrix_consensus.values, 0)
distance_matrix_consensus.head()

gene,ACE,ACOT11,ACSF2,ADAM11,ADAMTS12,ADCY1,ADCY4,AFAP1L1,ALDH1L1,ALOX12,...,WDR69,WFDC5,WIPF1,WNT16,ZFP42,ZNF167,ZNF48,ZNF577,ZNF655,ZP4
gene,,,,,,,,,,,,,,,,,,,,,
ACE,0.000000,0.666667,0.666667,0.000000,0.333333,0.333333,0.666667,0.666667,0.666667,0.666667,...,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667
ACOT11,0.666667,0.000000,0.000000,0.666667,0.666667,0.666667,0.333333,0.333333,0.333333,0.000000,...,0.666667,0.666667,0.333333,0.666667,0.333333,0.333333,0.333333,0.333333,0.333333,0.666667
ACSF2,0.666667,0.000000,0.000000,0.666667,0.666667,0.666667,0.333333,0.333333,0.333333,0.000000,...,0.666667,0.666667,0.333333,0.666667,0.333333,0.333333,0.333333,0.333333,0.333333,0.666667
ADAM11,0.000000,0.666667,0.666667,0.000000,0.333333,0.333333,0.666667,0.666667,0.666667,0.666667,...,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667
ADAMTS12,0.333333,0.666667,0.666667,0.333333,0.000000,0.666667,0.666667,0.666667,0.666667,0.666667,...,0.333333,0.333333,0.666667,0.333333,0.666667,0.666667,0.666667,0.666667,0.666667,0.333333


In [55]:
cluster_result_consensus = hierarchical_clustering(
    distance_matrix_consensus,
    range_min=2,
    range_max=4,
    cluster_number=4,
    out_path=f"{result_out_path}/hierarchical_clustering_consensus.png",
)

chosen number of clusters: 4


In [56]:
cluster_result_consensus.head()

,gene,cluster
0,ACE,1
1,ACOT11,4
2,ACSF2,4
3,ADAM11,1
4,ADAMTS12,3


#### 5. Compare


In [57]:
from utils.clustering_helper import hierarchical_clustering_compare

hierarchical_clustering_compare(
    [weighted_sum_dataframe, simple_sum_dataframe, distance_matrix_consensus],
    ["Weighted Average", "Simple Average", "Consensus"],
    out_path=f"{result_out_path}/hierarchical_clustering_compare.png",
    range_min=2,
    range_max=4,
)

Best number of clusters for Weighted Average: 3
Best number of clusters for Simple Average: 2
Best number of clusters for Consensus: 4


In [58]:
dbeta_info = pd.read_csv(dbeta_file)

In [59]:
# column gene isin weighted_sum_dataframe
weighted_dbeta = dbeta_info[dbeta_info["gene"].isin(weighted_sum_dataframe.index)]
simple_dbeta = dbeta_info[dbeta_info["gene"].isin(simple_sum_dataframe.index)]
consensus_dbeta = dbeta_info[dbeta_info["gene"].isin(distance_matrix_consensus.index)]

In [60]:
weighted_dbeta.merge(cluster_result_weighted, on="gene").to_csv(
    f"{result_out_path}/{result_prefix}_weighted.csv", index=False
)
simple_dbeta.merge(cluster_result_simple, on="gene").to_csv(
    f"{result_out_path}/{result_prefix}_simple.csv", index=False
)
consensus_dbeta.merge(cluster_result_consensus, on="gene").to_csv(
    f"{result_out_path}/{result_prefix}_consensus.csv", index=False
)

### Sec. 6 SimpleModel Training


#### 1. Load Data


In [89]:
from utils.train_helper import read_selected_features_json
import pandas as pd
from utils.process_norm import *

In [94]:
dbeta_file = config["simple_model"]["hyper"]["dbeta_file"]
selected_feature_file = config["simple_model"]["hyper"]["selected_feature_file"]
train_out_path = config["simple_model"]["hyper"]["train_out_path"]
validate_out_path = config["simple_model"]["hyper"]["validate_out_path"]
df_file = config["simple_model"]["hyper"]["df_file"]
training_param_file = config["simple_model"]["hyper"]["training_param_file"]

In [95]:
selected_feature_file = read_selected_features_json(selected_feature_file)
dbeta_file = pd.read_csv(dbeta_file)

In [96]:
all_df = pd.read_csv(df_file)

In [100]:
import json
with open(f"../config/{TYPE}.json") as f:
    J = json.load(f)
data_source = "GDC_prostate_tissue"
# data_source = input("Enter the data source: ")

all_df = organize_dataset(all_df, J[data_source]["normal"], J[data_source]["tumor"], J[data_source]["sample_count"])

In [101]:
sp80, sp20 = split_dataset(all_df, 0.2, 42)

INFO complement_df feature: 373340
INFO complement_df sample (normal, tumor): (401, 402)
INFO ratio_df feature: 373340
INFO ratio_df sample (normal, tumor): (101, 100)


In [102]:
sp80

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,793,794,795,796,797,798,799,800,801,802
0,cg00000957,0.824770,0.838333,0.875196,0.810238,0.860323,0.884161,0.864832,0.863311,0.861795,...,0.833749,0.886146,0.832414,0.892412,0.891901,0.905091,0.835944,0.745082,0.884742,0.869875
1,cg00001349,0.827872,0.875523,0.810576,0.806504,0.821141,0.852266,0.846344,0.715566,0.801538,...,0.605978,0.900709,0.819733,0.905889,0.931324,0.947951,0.130110,0.801109,0.833059,0.840674
2,cg00001583,0.121659,0.520205,0.241731,0.134270,0.351876,0.360133,0.258585,0.095149,0.048460,...,0.128480,0.286445,0.421377,0.622970,0.439232,0.783077,0.473451,0.067169,0.208544,0.460007
3,cg00002028,0.042886,0.034299,0.021822,0.020703,0.024398,0.017322,0.032324,0.028100,0.027861,...,0.040477,0.033830,0.043285,0.026778,0.023449,0.021500,0.034437,0.025156,0.024711,0.065117
4,cg00002719,0.065884,0.490811,0.069449,0.047640,0.212139,0.024730,0.206436,0.032892,0.048767,...,0.090535,0.376104,0.235849,0.807773,0.753019,0.814458,0.313751,0.151836,0.192325,0.494103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373336,cg27657363,0.945175,0.931059,0.957842,0.963020,0.966025,0.968234,0.967821,0.927369,0.951071,...,0.899267,0.960262,0.915603,0.946284,0.967195,0.916364,0.923337,0.950343,0.960460,0.902534
373337,cg27657537,0.298178,0.282966,0.245997,0.220096,0.283525,0.185075,0.381661,0.249224,0.101864,...,0.183660,0.427111,0.451669,0.736287,0.657288,0.753574,0.441521,0.151265,0.469957,0.293109
373338,cg27662611,0.068914,0.033763,0.043791,0.031257,0.060840,0.031299,0.067576,0.065646,0.063771,...,0.061230,0.030288,0.050947,0.065393,0.077492,0.064744,0.029955,0.055340,0.045296,0.052520
373339,cg27665648,0.858845,0.824831,0.880827,0.896936,0.942436,0.915946,0.922258,0.893683,0.925910,...,0.857805,0.918376,0.871963,0.753748,0.872375,0.925096,0.846256,0.802507,0.810491,0.906675


In [103]:
sp20

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,191,192,193,194,195,196,197,198,199,200
0,cg00000957,0.866258,0.848018,0.843079,0.830946,0.839486,0.807396,0.849108,0.858916,0.849391,...,0.809305,0.867759,0.880241,0.876851,0.874233,0.895410,0.850494,0.899322,0.880528,0.837177
1,cg00001349,0.852034,0.868409,0.854832,0.757838,0.832331,0.815491,0.731017,0.818800,0.734660,...,0.840027,0.793202,0.843819,0.811798,0.842095,0.802152,0.752804,0.866580,0.857917,0.792160
2,cg00001583,0.360523,0.103146,0.105064,0.050091,0.215267,0.136135,0.052770,0.224399,0.255017,...,0.133560,0.496975,0.481672,0.406983,0.625404,0.095873,0.276501,0.794870,0.838029,0.583207
3,cg00002028,0.020525,0.028840,0.025378,0.029126,0.025493,0.019812,0.027149,0.037735,0.036445,...,0.018602,0.014319,0.031543,0.035377,0.043325,0.016920,0.019389,0.046088,0.044256,0.025010
4,cg00002719,0.029307,0.051110,0.032113,0.035411,0.094550,0.047173,0.039447,0.088366,0.048843,...,0.345822,0.675067,0.696279,0.235251,0.574570,0.204156,0.025527,0.754098,0.850508,0.520944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373336,cg27657363,0.968167,0.957530,0.965772,0.946106,0.961166,0.964216,0.948009,0.966098,0.950632,...,0.934152,0.945786,0.897782,0.958972,0.944172,0.966352,0.958431,0.953648,0.920626,0.924836
373337,cg27657537,0.220613,0.168620,0.130816,0.109222,0.342180,0.232905,0.102387,0.313383,0.263290,...,0.411657,0.564375,0.665107,0.538359,0.610849,0.566487,0.153585,0.667166,0.729365,0.496844
373338,cg27662611,0.049505,0.044676,0.033138,0.058121,0.019983,0.029528,0.056332,0.068099,0.041418,...,0.036843,0.018571,0.035393,0.042450,0.064249,0.022017,0.044757,0.055987,0.042192,0.043591
373339,cg27665648,0.917473,0.920424,0.874715,0.879550,0.887804,0.899420,0.915299,0.917810,0.884160,...,0.813484,0.813176,0.897317,0.897426,0.870542,0.895949,0.878483,0.914036,0.875042,0.869568


#### 2. Training


In [104]:
from utils.train_helper import set_parameters
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

import json
from sklearn.ensemble import VotingClassifier


with open(training_param_file, "r") as f:
    training_param = json.load(f)

xgb_grid = set_parameters(XGBClassifier(random_state=42), training_param["XGBoost"])
rf_grid = set_parameters(RandomForestClassifier(random_state=42), training_param["RandomForest"])
svm_grid = set_parameters(SVC(random_state=42, probability=True), training_param["SVM"])
dt_grid = set_parameters(DecisionTreeClassifier(random_state=42), training_param["DecisionTree"])
voting = VotingClassifier(
    estimators=[("XGBoost", XGBClassifier(random_state=42)), ("RandomForest", RandomForestClassifier(random_state=42)), ("SVM", SVC(random_state=42, probability=True)), ("DecisionTree", DecisionTreeClassifier(random_state=42))
                ],
    voting="soft",
)

# comment out the model you don't want to use
models = {
    "XGBoost": {
        "is_grid_search": True,
        "model": xgb_grid,
    },
    "RandomForest": {
        "is_grid_search": True,
        "model": rf_grid,
    },
    "SVM": {
        "is_grid_search": True,
        "model": svm_grid,
    },
    "DecisionTree": {
        "is_grid_search": True,
        "model": dt_grid,
    },
    "Voting": {
        "is_grid_search": False,
        "model": voting,
    },
}

In [105]:
from utils.simple_model import SimpleModel

for model_name, gene_list in selected_feature_file.items():
    for model_name, model_config in models.items():
        model = SimpleModel(
            train_df=sp80,
            test_df=sp20,
            gene_list=gene_list,
            dbeta_info=dbeta_file,
        )
        model.setup_dbeta()
        model.setup_train_test()
        model.setup_combinations()
        model.train(
            model_name,
            model_config["model"],
            train_out_path,
            validate_out_path,
            model_config["is_grid_search"],
        )

INFO Training for combination: ('CDX2', 'ITIH5', 'OR4K2', 'RARRES1') with estimator: XGBoost
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
INFO Training for combination: ('GCM2', 'ITIH5', 'OR4K2', 'RARRES1') with estimator: XGBoost
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
INFO Training for combination: ('HAAO', 'ITIH5', 'OR4K2', 'RARRES1') with estimator: XGBoost
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
INFO Training for combination: ('RNF152', 'ITIH5', 'OR4K2', 'RARRES1') with estimator: XGBoost
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
INFO Training for combination: ('CDX2', 'ITIH5', 'OR4K2', 'RARRES1') with estimator: RandomForest
Fitting 5 folds for each of 27 candidates, totalling 135 fits
INFO Training for combination: ('GCM2', 'ITIH5', 'OR4K2', 'RARRES1') with estimator: RandomForest
Fitting 5 folds for each of 27 candidates, totalling 135 fits
INFO Training for combination: ('HAAO', 'ITIH5', '

#### 3. Visualization


In [104]:
model = input("Enter the model name: ")


##### Congratulation! You have finished the whole pipeline🎉. <br>

![title](cat.png)
